# DeepLearning Hate Speech
This notebook contains our project implementation, the instructions to run the notebook and explanations.

#### 1.1 Imports

In [1]:
import os
import shutil

In [2]:
# Define the directory paths

image_dir  = "/hateful_meme/img/"
source_dir = "/hateful_meme/"
destination_dir = "/path/to/destination_directory"